# Install Required Libraries

In [8]:
%pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain_text_splitters

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ.get['LANGCHAIN_TRACING_V2']
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = <your-api-key>

In [1]:
from utils.convert_to_md import convert_all_pdfs_to_md
from utils.sentence_chunking import get_sentence_chunks

import chromadb
from pathlib import Path
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter as Rec
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

splitted_chunks = []

# Initialize ChromaDB client
# client = chromadb.Client()
client = chromadb.PersistentClient(path="db/pdfs")


# Create a collection in ChromaDB (will store embeddings)
collection = client.get_or_create_collection(name="markdown_chunks_collection")

def process_all_markdowns_in_chroma():
    try:
        # Path to markdown directory
        md_dir = Path("data/md/")
        chunk_id_counter = 0  # Initialize a counter for unique chunk IDs

        # Loop through all markdown files in the md directory
        for md_file in md_dir.glob("*.md"):
            with open(md_file, "r") as f:
                md_content = f.read()

            # Chunk the markdown content
            ## Chunk Method 1: Sentence Chunking
            # chunks = get_sentence_chunks(md_content, tokenizer)
            
            ## Chunk Method 2: CST Token Chunking
            # chunks = get_cst_token_chunks(md_content, tokenizer)
            
            ## Chunk Method 3: Recursive Character Chunking
            text_splitter = Rec(
                chunk_size=1000,
                chunk_overlap=500,
                length_function=len,
                add_start_index=True
            )
            chunks = text_splitter.split_text(md_content)

            # Embed and add each chunk to ChromaDB
            for chunk in chunks:
                # Embed the chunk
                chunk_embedding = model.encode(chunk)

                # Use a standardized format for the source path (absolute path)
                source_path = str(md_file)

                # Add the chunk and its embedding to ChromaDB
                collection.add(
                    documents=[chunk],  # Pass chunk as a list
                    embeddings=chunk_embedding.tolist(),  # Pass embedding as a list of lists
                    metadatas={"source": source_path},
                    ids=str(chunk_id_counter)  # Use a unique integer as the ID
                )
                chunk_id_counter += 1  # Increment the ID counter

            print(f"Processed and stored {len(chunks)} chunks from {md_file}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # Convert all PDFs to markdown
    convert_all_pdfs_to_md()

    # Process and store all markdown files in ChromaDB
    process_all_markdowns_in_chroma()

c:\Users\elroy\anaconda3\envs\streamlitenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checking for punkt data in C:\Users\elroy/nltk_data\tokenizers\punkt
punkt data found in C:\Users\elroy/nltk_data\tokenizers\punkt
Checking for punkt_tab data in C:\Users\elroy/nltk_data\tokenizers\punkt_tab
punkt_tab data found in C:\Users\elroy/nltk_data\tokenizers\punkt_tab


c:\Users\elroy\anaconda3\envs\streamlitenv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing data\books\cp1.pdf...
[                                        ] (0/15[                                        ] (  1/15[                                        ] (  2/15[                                        ] (  3/152[=                                       ] (  4/152[=                                       ] (  5/152[=                                       ] (  6/1[=                                       ] (  7/15[==                                      ] (  8/15[==                                      ] (  9/15[==                                      ] ( 10/15[==                                      ] ( 11/152[===                                     ] ( 12/1[===                                     ] ( 13/152[===                                     ] ( 14/1[===                                     ] ( 15/15[====                                    ] ( 16/15[====                                    ] ( 17/15[====                                    ] ( 18/15[=====           

# Document Processing and Embedding with ChromaDB and Sentence Transformers

In [5]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path="db/pdfs")
md_collection = client.get_or_create_collection("markdown_chunks_collection")

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

### QUERY ###
query = "How does the insertion sort algorithm work?"

sentence = model.encode(query)

results = md_collection.query(
    query_embeddings = sentence.tolist(),
    n_results = 10
)

for i, document in enumerate(results["documents"][0]):
    print(f"Result {i+1}: {document}")
    print(f"ID: {results['ids'][0][i]}")
    # print(f"Metadata: {results['metadatas'][0][i]}")
    # print(f"Similarity score: {results['distances'][0][i]}\n")

Result 1: _CHAPTER 8. SORTING_ 67

###### 8.4 Insertion Sort

Insertion sort is a somewhat interesting algorithm with an expensive runtime of
_O(n[2]). It can be best thought of as a sorting scheme similar to that of sorting_
a hand of playing cards, i.e. you take one card and then look at the rest with
the intent of building up an ordered set of cards in your hand.

4 75 74

4 75 74 2 54 4 75 74 2 54 4 75 74 2 54

2 54

4 74 75 2 54 2 4 74 75 54 2 4 54 74 75

Figure 8.4: Insertion Sort Iterations

1) algorithm Insertionsort(list)
2) **Pre:** _list_ =
_̸_ _∅_
3) **Post: list has been sorted into values of ascending order**
4) _unsorted_ 1
_←_
5) **while unsorted < list.Count**
6) _hold_ _list[unsorted]_
_←_
7) _i_ _unsorted_ 1
_←_ _−_
8) **while i** 0 and hold < list[i]
_≥_
9) _list[i + 1]_ _list[i]_
_←_
10) _i_ _i_ 1
_←_ _−_
11) **end while**
12) _list[i + 1]_ _hold_
_←_
13) _unsorted_ _unsorted + 1_
_←_
14) **end while**
15) **return list**
16) end Insertionsort

|4|Col2|
|---|---|
|

In [2]:
results2 = md_collection.get()

gathered_chunks = []
for idx, chunk in enumerate(results2['documents']):
    # print(chunk[0])
    gathered_chunks.append(chunk)
    

In [3]:
from utils.bm25_ranking import find_closest_chunks_bm25, new_bm25, re_rank_chunks_with_embeddings
print("OLD BM25")
results_top_n = find_closest_chunks_bm25(query, results2, top_n=10)
for res in results_top_n:
    print(res['score']," ",res['id'],"\n")
    # print(res['document'],"\n")


print("NEW BM25")
bm25_results = new_bm25(query, results2, top_n=10)
sorted_results = re_rank_chunks_with_embeddings(query, bm25_results)

# print(results_top_n)
# for res in results_top_n:
#     print(res['score']," ",res['id'],"\n")
#     print(res['document'],"\n")

# organised_list = {}
# for item in results2:
#     print(item)
#     # organised_list[results2[item]] = ## id and document
# print(organised_list)
# print(results_top_n)
# print(md_collection.get(ids="297"))

OLD BM25
14.584834019480919   377 

12.523017371922746   378 

12.050891594269249   375 

11.880403985989389   866 

11.260478820127473   535 

11.123203021744485   815 

11.035082004738484   814 

11.002614755875888   406 

10.965935890038589   260 

10.352130470797842   805 

NEW BM25
Chunk ID: 815, BM25 Score: 11.123203021744485, Embedding Score: 0.4674035310745239, Source: data\md\dsa.md
Most libraries provide implementations of unordered sets and so DSA does
not; we simply mention it here to disambiguate between an unordered set and
ordered set.

We will only look at insertion for an unordered set and cover briefly why a
hash table is an efficient data structure to use for its implementation.

###### 5.1.1 Insertion

An unordered set can be efficiently implemented using a hash table as its backing
data structure. As mentioned previously we only add an item to a set if that
item is not already in the set, so the backing data structure we use must have
a quick look up and insertion 

In [22]:
print(md_collection.get(ids=['220']))

{'ids': ['220'], 'embeddings': None, 'metadatas': [{'source': 'data\\md\\cp1.md'}], 'documents': ['best possible acorns collected when Jayjay is at this height. The bottom-up DP code that requires\n\nonly 2000 = 2K states and time complexity of 2000 2000 = 4M is as follow:\n_×_\n\nfor (int tree = 0; tree < t; tree++) // initialization\ndp[h] = max(dp[h], acorn[tree][h]);\nfor (int height = h - 1; height >= 0; height--)\nfor (int tree = 0; tree < t; tree++) {\nacorn[tree][height] +=\nmax(acorn[tree][height + 1], // from this tree, +1 above\n((height + f <= h) ? dp[height + f] : 0)); // best from tree at height + f\ndp[height] = max(dp[height], acorn[tree][height]); // update this too\n}\nprintf("%d\\n", dp[0]); // solution will be here\n\nLesson: When na¨ıve DP states are too large causing the overall DP time complexity not-doable,\n\nthink of different ways other than the obvious to represent the possible states. Remember that no\n\nprogramming contest problem is unsolvable, the proble